In [7]:
from parsing.parse_functions import parse_pdb_files
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D   # noqa: F401  (needed for 3-D projection)
from scipy.cluster.hierarchy import average as average_linkage
from scipy.cluster.hierarchy import single as single_linkage
from scipy.cluster.hierarchy import fcluster
# for computing the distance matrix:
from scipy.spatial.distance import pdist, squareform
import pickle
import os

from utils.scale_low_res_coordinates import scale_low_res_coords
from utils.pucker_data_functions import determine_pucker_data
from shape_analysis import pre_clustering
from utils.pucker_data_functions import sort_data_into_cluster
from pnds.PNDS_RNA_clustering import new_multi_slink
from utils import plot_functions

In [2]:
input_pdb_dir = "/Users/kaisardauletbek/Documents/GitHub/RNA-Classification/data/rna2020_pruned_pdbs/"
suites = parse_pdb_files(input_pdb_dir, input_pdb_folder=input_pdb_dir)

puckers = ['c2c2', 'c2c3', 'c3c2', 'c3c3']
pucker_indices = {}
for pucker in puckers:
    indices, _ = determine_pucker_data(suites, pucker)
    pucker_indices[pucker] = indices

# -- ensure every index array is really integer-typed  -----------------
pucker_indices = {k: np.asarray(v, dtype=np.intp)    # <-- np.intp = “platform int”
                  for k, v in pucker_indices.items()}


In [3]:
pucker_indices

{'c2c2': array([  16,   17,   52,   84,  174,  179,  202,  206,  308,  309,  341,
         500,  563,  574,  599,  642,  651,  652,  666,  800,  806,  807,
         808,  831,  841,  905, 1077, 1120, 1152, 1262, 1287, 1291, 1293,
        1294, 1312, 1313, 1352, 1394, 1395, 1396, 1417, 1458, 1476, 1477,
        1521, 1522, 1524, 1547, 1548, 1549, 1550, 1572, 1601, 1602, 1603,
        1610, 1694, 1764, 1765, 1772, 1789, 1842, 1846, 1847, 1852, 1928,
        1984, 1990, 2038, 2093, 2094, 2100, 2116, 2123, 2175, 2191, 2210,
        2238, 2257, 2258, 2277, 2278, 2279, 2285, 2307, 2316, 2317, 2330,
        2332, 2336, 2342, 2403, 2404, 2410, 2442, 2443, 2491, 2503, 2504,
        2535, 2545, 2548, 2549, 2578, 2676, 2758, 2926, 2932, 2938, 2939,
        2982, 3051, 3060, 3097, 3116, 3138, 3242, 3267, 3268, 3319, 3444,
        3445, 3446, 3539, 3592, 3645, 3646, 3682, 3683, 3684, 3874, 3875,
        3888, 3909, 3919, 3930, 3958, 4131]),
 'c2c3': array([  53,   64,   75,   85,   96,  172,  177, 

In [4]:
# ---------------------------------------------------------------------------
# helpers
# ---------------------------------------------------------------------------
def spherical_to_vec(theta_deg: np.ndarray, phi_deg: np.ndarray) -> np.ndarray:
    t, p = np.radians(theta_deg), np.radians(phi_deg)
    return np.column_stack([np.sin(t) * np.cos(p),
                            np.sin(t) * np.sin(p),
                            np.cos(t)])

def arc_distance(a: np.ndarray, b: float) -> np.ndarray:
    """Shortest signed arc distance between vectors of angles a and scalar b (radians)."""
    d = (a - b + np.pi) % (2 * np.pi) - np.pi
    return d

def exponential_map(V, p):
    """
    V - point cloud N x R^m
    p - point of tangency R^m
    """
    N, M = V.shape[0], V.shape[1]
    V_mean = V.mean(axis=0)
    # check if points are centered at 0, center them
    V -= V_mean
    V = np.column_stack([V, np.zeros(N)])
    V_norm = np.linalg.norm(V, axis=1)[:, None]
    
    return np.cos(V_norm) * p + np.sin(V_norm) * (V / V_norm)
    

In [5]:
# scale only the distance‐variance, leave α‐variance and both means alone
scaled_coords, lambda_d, lambda_alpha = scale_low_res_coords(
    suites,
    scale_distance_variance=True,
    scale_alpha_variance=False,
    preserve_distance_mean=True,
    preserve_alpha_mean=True,
    store_attr="scaled_dvar_only"
)

d2_s, d3_s, alpha_s, theta1, phi1, theta2, phi2 = scaled_coords.T
N = len(d2_s)

In [6]:
x = np.array([d2_s[0], d3_s[0], alpha_s[0], theta1[0], phi1[0], theta2[0], phi2[0]])
x_prime = np.array([d2_s[1], d3_s[1], alpha_s[1], theta1[1], phi1[1], theta2[1], phi2[1]])

In [7]:
np.dot(x, x_prime)

20242.6872313057

In [8]:
# def get_distance(x, x_prime):
#     return np.dot(x, x_prime)

# def get_distance_matrix(scaled_coords):
#     d2_s, d3_s, alpha_s, theta1, phi1, theta2, phi2 = scaled_coords.T
#     N = len(d2_s)
#     distance_matrix = np.zeros((N, N))
#     for i in range(N):
#         for j in range(i+1, N):
#             x = np.array([d2_s[i], d3_s[i], alpha_s[i], theta1[i], phi1[i], theta2[i], phi2[i]])
#             x_prime = np.array([d2_s[j], d3_s[j], d3_s[j], theta1[j], phi1[j], theta2[j], phi2[j]])
#             distance_matrix[i, j] = get_distance(x, x_prime)
#     return distance_matrix

# # distance_matrix = get_distance_matrix(scaled_coords)


In [9]:
scaled_coords[pucker_indices['c3c3']].shape

(3585, 7)

In [8]:
names = ['c3c3', 'c3c2', 'c2c3', 'c2c2', 'all']
min_size = 3

configurations = [
    ('c2c2', 0.02, 0.05),
    ('c2c3', 0.02, 0.07),
    ('c3c2', 0.02, 0.05),
    ('c3c3', 0.02, 0.09),
]

# Folder to store preclustering results
os.makedirs('preclustering_results', exist_ok=True)

for name, outlier, qf in configurations:
    scaled_coords_by_pucker = scaled_coords[pucker_indices[name]]
    
    clusters, outliers, _ = pre_clustering(
        input_data = scaled_coords_by_pucker,
        m = min_size,
        percentage = 0.0,  # or outlier if needed
        string_folder = "out/clustering_low_res",
        method = average_linkage,
        q_fold = qf,
        distance = "low_res_suite_shape"
    )

    # Save precluster results to .pkl
    result = {
        'name': name,
        'clusters': clusters,
        'outliers': outliers,
        'scaled_coords': scaled_coords_by_pucker
    }
    with open(f'preclustering_results/{name}_precluster.pkl', 'wb') as f:
        pickle.dump(result, f)

    # Optional: also plot & inspect
    sorted_clusters = sorted(clusters, key=len, reverse=True)
    cluster_sizes = [len(c) for c in sorted_clusters]
    print(f"[{name}] cluster sizes:", cluster_sizes)

    data_by_cluster, _ = sort_data_into_cluster(scaled_coords_by_pucker, sorted_clusters, min_size)
    plot_functions.scatter_plots(
        data_by_cluster,
        filename=f"low_res_suite_shape_average_linkage_{name}",
        set_title=f"low_res_suite_shape_average_linkage {name}",
        suite_titles=[r'$d_2$', r'$d_3$', r'$\alpha$', r'$\theta_1$', r'$\phi_1$', r'$\theta_2$', r'$\phi_2$'],
        list_ranges=[[3,5.5],[4,6.5],[0,180],[0,180],[-180,180],[0,180],[-180,180]],
        number_of_elements=cluster_sizes,
        legend=True,
        s=20
    )


no shape_space
138
12.12435565298214
87
9.797958971132712
64
8.54400374531753
38
6.855654600401044
12
4.58257569495584
3.6203055098180523
[c2c2] cluster sizes: [51, 26, 26, 23, 12]
no shape_space
221
15.165750888103101
169
13.341664064126334
124
11.532562594670797
87
9.797958971132712
51
7.745966692414834
26
5.916079783099616
15
4.898979485566356
6
3.872983346207417
3.4592907592173328
[c2c3] cluster sizes: [52, 45, 37, 36, 25, 11, 9, 6]
no shape_space
269
16.673332000533065
209
14.7648230602334
166
13.228756555322953
132
11.874342087037917
99
10.392304845413264
70
8.888194417315589
37
6.782329983125268
23
5.656854249492381
10
4.358898943540674
3.705951988674172
[c3c2] cluster sizes: [60, 43, 34, 33, 33, 29, 14, 13, 10]
no shape_space
3585
59.94997914928745
3447
58.787753826796276
3309
57.60208329565867
2352
48.590122453025366
467
21.817424229271428
393
20.049937655763422
200
14.45683229480096
159
12.96148139681572
118
11.269427669584644
60
8.306623862918075
31
6.324555320336759
3.77214

In [ ]:
min_size = 3
scale = 12000

# Where you saved them
result_dir = 'preclustering_results'

mode_clusters_res = []

for filename in os.listdir(result_dir):
    if filename.endswith('_precluster.pkl'):
        # Load
        with open(os.path.join(result_dir, filename), 'rb') as f:
            result = pickle.load(f)

        name = result['name']
        clusters = result['clusters']
        outliers = result['outliers']
        scaled_coords_by_pucker = result['scaled_coords']

        # Map to sphere and angles for PNS input
        d2_s, d3_s, alpha_s, theta1, phi1, theta2, phi2 = scaled_coords_by_pucker.T
        V = np.column_stack([d2_s, d3_s])
        S2_pts = exponential_map(V, p=np.array([0,0,1]))
        x, y, z = S2_pts.T

        r = np.linalg.norm(S2_pts, axis=1)
        theta_e_deg = np.degrees(np.arccos(np.clip(z/r, -1, 1)))
        phi_e_deg = (np.degrees(np.arctan2(y, x)) + 360.0) % 360.0

        angle_matrix = np.column_stack([
            theta_e_deg,
            phi_e_deg,
            alpha_s,
            theta1,
            phi1,
            theta2,
            phi2
        ])

        # Run PNS-based clustering
        mode_clusters, _ = new_multi_slink(
            scale=scale,
            data=angle_matrix,
            cluster_list=clusters,
            outlier_list=outliers,
            min_cluster_size=min_size
        )

        mode_clusters_res.append({
            'name': name,
            'mode_clusters': mode_clusters
        })

        print(f"[{name}] mode clusters done: {len(mode_clusters)} clusters")

# You can save final mode_clusters_res if needed
with open('mode_clusters_results.pkl', 'wb') as f:
    pickle.dump(mode_clusters_res, f)


In [ ]:
# names = ['c3c3', 'c3c2', 'c2c3', 'c2c2', 'all']
# min_size = 3
# configurations = [
#     ('c2c2', 0.02, 0.05),
#     ('c2c3', 0.02, 0.07),
#     ('c3c2', 0.02, 0.05),
#     ('c3c3', 0.02, 0.09),
# ]
# # try 0.001 increment for qf
# precluster_results = []
# mode_clusters_res = []
# for name, outlier, qf in configurations:
#     scaled_coords_by_pucker = scaled_coords[pucker_indices[name]]
#     clusters, outliers, _ = pre_clustering(
#     input_data = scaled_coords_by_pucker,
#     m = min_size, # min_cluster_size
#     percentage = 0.0,#outlier, # for d_max computation
#     string_folder = "out/clustering_low_res",
#     # method = average_linkage,
#     method = average_linkage,
#     q_fold = qf,
#     distance = "low_res_suite_shape"
#     )
#     precluster_results.append({
#         'name': name,
#         'clusters': clusters,
#         'outliers': outliers,
#         'scaled_coords': scaled_coords_by_pucker
#     })
#     sorted_clusters = clusters
#     sorted_clusters = sorted(sorted_clusters, key=len, reverse=True)
#     cluster_sizes = [len(c) for c in sorted_clusters]
#     print(cluster_sizes)
#     data_by_cluster, _ = sort_data_into_cluster(scaled_coords_by_pucker, sorted_clusters, min_size)
#     # Plot dihedral clusters
#     qfold_dir = "out/clustering_low_res/qfold_plots"
#     # ensure_dir(qfold_dir)
#     plot_functions.scatter_plots(
#         data_by_cluster,
#         filename="low_res_suite_shape_average_linkage_" + name,
#         set_title=f"low_res_suite_shape_average_linkage {name}",
#         suite_titles=[r'$d_2$',r'$d_3$',r'$\alpha$',r'$\theta_1$',r'$\phi_1$',r'$\theta_2$',r'$\phi_2$'],
#         list_ranges=[[3,5.5],[4,6.5],[0,180],[0,180],[-180,180],[0,180],[-180,180]],
#         number_of_elements=cluster_sizes,
#         legend=True,
#         s=20,
#         # legend_with_clustersize=True,
#     )

#         # scatter_plots(np.vstack(low_resolution_shape_list), filename='c3c3-low-res',
#         #           number_of_elements = len_list,
#         #           suite_titles=[r'$d_2$',r'$d_3$',r'$\alpha$',r'$\theta_1$',r'$\phi_1$',r'$\theta_2$',r'$\phi_2$'],
#         #           # color_and_marker_list=[['blue']*len(low_resolution_shapes_test),
#         #           #                        ['o']*len(low_resolution_shapes_test)],
#         #           list_ranges=[[3,5.5],[4,6.5],[0,180],[0,180],[-180,180],[0,180],[-180,180]],
#         #           s=60, fontsize=40)

#     d2_s, d3_s, alpha_s, theta1, phi1, theta2, phi2 = scaled_coords_by_pucker.T
#     V = np.column_stack([d2_s, d3_s])
#     S2_pts = exponential_map(V, p = np.array([0,0,1]))
#     x = S2_pts[:, 0]
#     y = S2_pts[:, 1]
#     z = S2_pts[:, 2]

#     # Compute the polar angle θ_e = arccos(z / r), then convert to degrees.
#     r = np.linalg.norm(S2_pts, axis=1)                     # Should be ≈1
#     theta_e_rad = np.arccos(np.clip(z / r, -1.0, 1.0))      # in radians
#     theta_e_deg = np.degrees(theta_e_rad)                  # in [0, 180]

#     # Compute the azimuth φ_e = atan2(y, x), then convert to [0, 360) degrees.
#     phi_e_rad = np.arctan2(y, x)                            # in (–π, π]
#     phi_e_deg = (np.degrees(phi_e_rad) + 360.0) % 360.0     # in [0, 360)

#     angle_matrix = np.column_stack([
#         theta_e_deg,  # shape (N,)
#         phi_e_deg,    # shape (N,)
#         alpha_s,      # shape (N,)
#         theta1,       # shape (N,)
#         phi1,         # shape (N,)
#         theta2,       # shape (N,)
#         phi2          # shape (N,)
#     ])
#     mode_clusters, _ = new_multi_slink(
#         scale=12000,
#         data = angle_matrix,
#         cluster_list=clusters,
#         outlier_list=outliers,
#         min_cluster_size=min_size
#     )
#     mode_clusters_res.append({
#         'name': name,
#         'mode_clusters': mode_clusters,
#     })




no shape_space
138
12.12435565298214
87
9.797958971132712
64
8.54400374531753
38
6.855654600401044
12
4.58257569495584
3.6203055098180523
[51, 26, 26, 23, 12]


In [7]:
names = ['c3c3', 'c3c2', 'c2c3', 'c2c2', 'all']
min_size = 3
configurations = [
    # ('c2c2', 0.02, 0.05),
    # ('c2c3', 0.02, 0.07),
    # ('c3c2', 0.02, 0.25),
    ('c3c3', 0.02, 0.35),
]

optimal_qfold_results = []

for name, outlier, initial_qf in configurations:
    q_fold = initial_qf
    scaled_coords_by_pucker = scaled_coords[pucker_indices[name]]

    while True:
        clusters, outliers, _ = pre_clustering(
            input_data=scaled_coords_by_pucker,
            m=min_size,
            percentage=0.0,
            string_folder="out/clustering_low_res",
            method=average_linkage,
            q_fold=q_fold,
            distance="low_res_suite_shape"
        )

        if len(clusters) == 5:
            optimal_qfold_results.append({
                'name': name,
                'optimal_q_fold': q_fold,
                'clusters': clusters,
                'outliers': outliers
            })
            print(f"Optimal q_fold for {name} found: {q_fold}")
            break

        print(f"Trying q_fold={q_fold} for {name}, found {len(clusters)} clusters")

        q_fold += 0.01

        if q_fold > 2:  # Arbitrary upper limit to avoid infinite loops
            print(f"No suitable q_fold found for {name}")
            break

no shape_space
3585
59.94997914928745
193
14.212670403551895
129
11.74734012447073
81
9.486832980505138
46
7.416198487095663
27
6.0
13
4.69041575982343
95.2738993531711
Trying q_fold=0.35 for c3c3, found 7 clusters
no shape_space


KeyboardInterrupt: 

'name': 'c3c2',
 'optimal_q_fold': 0.37,


 'name': 'c2c2',
 'optimal_q_fold': 0.31,

 'name': 'c3c2',
  'optimal_q_fold': 0.37,

q_fold=0.35 for c3c3, found 7 clusters

In [10]:
cluster_sizes = [len(c) for c in sorted_clusters]

In [86]:
scaled_coords_by_pucker

array([[ 4.77551022e+00,  5.19402460e+00,  6.56238856e+01, ...,
         2.00563389e+01,  1.00718394e+02, -2.02173404e-03],
       [ 4.86420256e+00,  5.62238196e+00,  6.61482139e+01, ...,
         2.52582240e+01,  1.00389302e+02,  7.30176475e+00],
       [ 4.70480812e+00,  5.29430901e+00,  6.83177486e+01, ...,
         2.25628491e+01,  1.13267129e+02,  5.46496333e+00],
       ...,
       [ 4.77173519e+00,  5.44072768e+00,  6.53622807e+01, ...,
         2.07444991e+01,  1.06416258e+02,  2.95847892e+00],
       [ 4.86873624e+00,  5.33394914e+00,  6.35468867e+01, ...,
         2.36774746e+01,  1.06502848e+02,  1.86457109e+00],
       [ 4.78531136e+00,  5.51888026e+00,  6.64054167e+01, ...,
         2.21053283e+01,  1.11631624e+02,  7.17826310e+00]])

In [9]:
d2_s, d3_s, alpha_s, theta1, phi1, theta2, phi2 = scaled_coords_by_pucker.T

In [10]:
alpha_s

array([65.62388563, 66.14821392, 68.3177486 , ..., 65.36228073,
       63.54688673, 66.40541669])

In [112]:
V = np.column_stack([d2_s, d3_s])
S2_pts = exponential_map(V, p = np.array([0,0,1]))
x = S2_pts[:, 0]
y = S2_pts[:, 1]
z = S2_pts[:, 2]

# Compute the polar angle θ_e = arccos(z / r), then convert to degrees.
r = np.linalg.norm(S2_pts, axis=1)                     # Should be ≈1
theta_e_rad = np.arccos(np.clip(z / r, -1.0, 1.0))      # in radians
theta_e_deg = np.degrees(theta_e_rad)                  # in [0, 180]

# Compute the azimuth φ_e = atan2(y, x), then convert to [0, 360) degrees.
phi_e_rad = np.arctan2(y, x)                            # in (–π, π]
phi_e_deg = (np.degrees(phi_e_rad) + 360.0) % 360.0     # in [0, 360)

angle_matrix = np.column_stack([
    theta_e_deg,  # shape (N,)
    phi_e_deg,    # shape (N,)
    alpha_s,      # shape (N,)
    theta1,       # shape (N,)
    phi1,         # shape (N,)
    theta2,       # shape (N,)
    phi2          # shape (N,)
])

In [113]:
angle_matrix.shape

(4213, 7)

In [114]:
from pnds.PNDS_geometry import RESHify_1D, unRESHify_1D

In [115]:
data_resh, cuts, half = RESHify_1D(angle_matrix, invert=True)

In [116]:
data_resh.shape

(4213, 8)

In [34]:
for res in precluster_results:
    print(res['name'], len(res['clusters']))

c2c2 4
c2c3 8
c3c2 9
c3c3 19
